### Airflow

Apache Airflow — это платформа для управления рабочими процессами (workflow), которая позволяет пользователям автоматизировать, планировать и мониторить сложные процессы обработки данных. 

Основные концепции
1. DAG (Directed Acyclic Graph):
    - DAG — это основной строительный блок в Airflow. Он представляет собой граф, в котором узлы — это задачи (tasks), а ребра — зависимости между ними. DAG всегда направленный и ациклический, что означает, что он не может содержать циклы.
    - DAG описывается в Python-файле, где вы определяете задачи и их зависимости.
2. Задачи (Tasks):
    - Задачи — это отдельные шаги в вашем рабочем процессе. Каждая задача может выполнять определенную операцию, такую как извлечение данных, их обработка или загрузка в хранилище.
    - Задачи могут быть реализованы с помощью операторов (operators), которые предоставляют готовые реализации для выполнения различных действий (например, BashOperator, PythonOperator, и т.д.).
3. Операторы (Operators):
    - Операторы — это абстракции, которые определяют, что будет делать задача. Например, BashOperator выполняет команды оболочки, а PythonOperator выполняет функции Python.
    - Airflow предоставляет множество встроенных операторов, а также возможность создавать свои собственные.
4. Планировщик (Scheduler):
    - Планировщик отвечает за выполнение задач в соответствии с расписанием, определенным в DAG. Он отслеживает состояние задач и запускает их, когда все зависимости выполнены.
5. Веб-интерфейс (Web UI):
    - Airflow предоставляет веб-интерфейс для мониторинга и управления DAG. Вы можете видеть состояние задач, их логи, а также запускать и останавливать DAG.
6. Экзекьюторы (Executors):
    - Экзекьюторы определяют, как задачи будут выполняться. Airflow поддерживает различные типы экзекьюторов, такие как SequentialExecutor, LocalExecutor, CeleryExecutor и другие, которые позволяют выполнять задачи локально или распределенно.

installation  
https://airflow.apache.org/docs/apache-airflow/stable/installation/index.html

https://habr.com/ru/articles/860900/

Через pip

In [ ]:
mkdir ~/airflow
cd ~/airflow
python -m venv airflow
source airflow/bin/activate 
# следующую строку лучше взять со свежей версией тут
# https://airflow.apache.org/docs/apache-airflow/stable/installation/installing-from-pypi.html
pip install "apache-airflow[celery]==2.10.4" --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.10.4/constraints-3.8.txt"

In [ ]:
# инициализации и миграции базы данных
airflow db migrate

In [ ]:
#Создание стандартных подключений: Если вам нужно создать стандартные подключения, используйте следующую команду
airflow connections create-default-connections

In [ ]:
# Создаем пользователя
airflow users create --username admin --password admin --firstname Firstname --lastname Lastname --role Admin --email your_email@gmail.com    

In [ ]:
# Запустите веб-сервер:
airflow webserver --port 8080

In [ ]:
# В другом терминале запустите планировщик:
airflow scheduler

Теперь вы можете получить доступ к веб-интерфейсу Airflow, открыв браузер и перейдя по адресу http://localhost:8080.

Вы можете настроить Airflow, изменив файл конфигурации airflow.cfg, который находится в директории ~/airflow/airflow.cfg.

Пример простого дага

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd

TOP_1M_DOMAINS = 'http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'

def get_data():
    top_doms = requests.get(TOP_1M_DOMAINS, stream=True)
    zipfile = ZipFile(BytesIO(top_doms.content))
    top_data = zipfile.read(TOP_1M_DOMAINS_FILE).decode('utf-8')
    with open (TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)

def get_stat_ru():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_top_10 = top_data_df[top_data_df.domain.str.endswith('.ru')].head(10)
    with open('top_data_top_10_ru.csv', 'w') as f:
        top_data_top_10.to_csv(f, index=False, header=False)


def get_stat_com():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_top_10 = top_data_df[top_data_df.domain.str.endswith('.com')].head(10)
    with open('top_data_top_10_com.csv', 'w') as f:
        top_data_top_10.to_csv(f, index=False, header=False)
# Если мы хотим передать параметр airflow в функцию, например, дату выполнения дага, то просто передаем ее в функцию, главное чтобы название параметра совпадало с параметром в airflow
def print_data(ts):
    with open('top_data_top_10_ru.csv', 'r') as f:
        all_data = f.read()
    print(f'Top 10 domains with .ru for date {ts}:')
    print(all_data)
    with open('top_data_top_10_com.csv', 'r') as f:
        all_data = f.read()
    print(f'Top 10 domains with .com for date {ts}:')
    print(all_data)

# Определяем параметры DAG
default_args = {
    'owner': 'admin',
    'depends_on_past': False,
    'start_date': datetime(2025, 1, 23),
    # 'end_date': datetime(2025, 1, 25),
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    # 'on_failure_callback': None, # тут можно прописать функцию для отправки сообщения в телеграм и сообщить информацию о падении дага
    # 'email_on_failure': True,
    # 'email': ['airflow@example.com'],
}

# Создаем DAG
dag = DAG(
    'top_10_ru',
    default_args=default_args,
    description='Dag for get top 10 domains with .ru',
    schedule_interval='0 12 * * *',  # можно задать через timedelta(days=1) и также через константы airflow @hourly, @daily, @weekly, @monthly
)

t1 = PythonOperator(python_callable=get_data, task_id='get_data', dag=dag)
t2_ru = PythonOperator(python_callable=get_stat_ru, task_id='get_stat_ru', dag=dag)
t2_com = PythonOperator(python_callable=get_stat_com, task_id='get_stat_com', dag=dag)
t3 = PythonOperator(python_callable=print_data, task_id='print_data', dag=dag)

t1 >> [t2_ru, t2_com] >> t3

# t1.set_downstream(t2_ru)
# t1.set_downstream(t2_com)
# t2_ru.set_downstream(t3)
# t2_com.set_downstream(t3)


Пример более сложного дага

### Cron

Планирование сценариев с помощью Сron

Cron это просто файл  
сам cron постоянно работает в unix  
добавил в него файл - он работает по расписанию  
удалил - перестал работать

Ниже приведен синтаксис для планирования crons:

In [ ]:
# Cron job example* * * * * sh /path/to/script.sh

Здесь * представляют собой минуты (минуты), часы (часы), день (дни), месяцы и дни недели соответственно.

Теперь настроим cron, чтобы автоматизировать процесс
cron это демон в линуксе, который может по расписанию запускать скрипты (по сути он выполняет по расписанию bash команды)  



crontab -e (создает файл, если его нет, если есть, то открывает на редактирование)  
crontab -l (открывает список задача в кроне)  
crontab -r (удаляет крон файл)  
crontab -v (показывает время, когда последний раз редактировался крон файл)  
Пользователь сам может выбрать подходящий для него формат, снимая или устанавливая "галочку" в опции "Поле года" (Year field)  
диалогового окна "Параметры" (двойной щелчок на иконке nnCron в системном трее). По умолчанию nnCron использует "улучшенный" формат.

Вот его схема:
https://nncron.ru/help/RU/working/cron-format.htm


На этом сайте можно проверить правильность указания времени для крона  
https://crontab.guru/

In [ ]:
* * * * * 
| | | | | 
| | | | | 
| | | | +----- Дни недели (диапазон: 0-6)
| | | +------- Месяцы     (диапазон: 1-12)
| | +--------- Дни месяца (диапазон: 1-31)
| +----------- Часы       (диапазон: 0-23)
+------------- Минуты     (диапазон: 0-59)    

In [ ]:
*	any value
,	value list separator
-	range of values
/	step values

Очень важный момент, не все команды, которые мы можем выполнить в баше в терминале запустятся в кроне,  
для них нужно указать полный путь до экзешника.  
Очевидное не всегда очевидно: простая команда вроде python3 не  
выполнится!  
Что делать? Посмотреть, где он лежит через which python3 и  
использовать полный путь.  

In [ ]:
Важно, если мы хотим указать точное время, то нужно указать нули во времени в предыдущих разрядах,  
нарпимер, чтобы запускался в 3 утра кадый день, мы не можем написать  
* 3 * * *
так будет с 3:00 до 3:59 запускатся каждую минуту, так как стоит * это любое количество  
поэтому нужно  
0 3 * * *

In [ ]:

создадим распсание крона
crontab -e
выбираем редактор, например, nano
# вначале пишем эту строчку, это значит каждую минуту будет запускаться эта команда
* * * * * python ~/cron_testing.py
сохраняем ctrl + o
можно протестиьть работу крона,  
для этого загрузим файл cron_testing.py на машину
# проверим, что все записалось
crontab -l
все ок, теперь можно смотреть в базе, что наш крон работает
теперь настроим крон, чтобы он ночью заливал данные
crontab -e
# в будет каждый день в 0:00 
0 0 * * * python3 ~/work.sh
# будет запускаться в субботу и воскресенье в 4 утра
0 4 * * 0,6 python3 ~/work.sh
# будет запускаться каждые 2 часа  
0 */2 * * *
# будет запускаться  1-го 5го 11го и 20го числа кадого месяца  
0 0 1, 5, 11, 20 * *


Пример использования с перенаправлением вываода

In [ ]:
# Это файл test.py

if __name__ == '__main__':
    print('a minute passed')

Следующая записи в крон файле будет каждую минуту запускаь файл `test.py` и перенаправлять вывод этого скприпта в лог  
(это просто пример, можно конечно сделать логер внутри скрипта, но может и так будет нужно)

Тут важно, что когда мы кроном запускаем скрипт, то сообщения о ошибках или прудупреждения питона потеряются  
Чтобы они логировались, мы перенаправляем стандартный вывод в файл и указываем, что ошибки также выводить туда

In [ ]:
* * * * * /opt/anaconda3/bin/python3 ~/scripts/test.py >> ~/log.txt 2>&1

In [ ]:
Символ 2>&1 значит
“направляй поток ошибок туда же, куда и
простой вывод”. Основной вывод здесь скрывается за константой 1, а
поток ошибок – 2.
Это приведет к тому, что в одном месте будут собраны все логи, – а тогото мы и добивались.